# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-06-18 00:49:48] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=39539, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=228634466, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1

[2025-06-18 00:50:03] Attention backend not set. Use fa3 backend by default.
[2025-06-18 00:50:03] Init torch distributed begin.


[2025-06-18 00:50:04] Init torch distributed ends. mem usage=0.00 GB


[2025-06-18 00:50:05] Load weight begin. avail mem=77.30 GB


[2025-06-18 00:50:06] Using model weights format ['*.safetensors']


...

[2025-06-18 00:50:07] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-06-18 00:50:07] INFO:     Started server process [1302605]
[2025-06-18 00:50:07] INFO:     Waiting for application startup.
[2025-06-18 00:50:07] INFO:     Application startup complete.
[2025-06-18 00:50:07] INFO:     Uvicorn running on http://0.0.0.0:39539 (Press CTRL+C to quit)


[2025-06-18 00:50:08] INFO:     127.0.0.1:41398 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-18 00:50:08] INFO:     127.0.0.1:41400 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-18 00:50:08] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-18 00:50:09] INFO:     127.0.0.1:41410 - "POST /generate HTTP/1.1" 200 OK
[2025-06-18 00:50:09] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-18 00:50:13] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-18 00:50:13] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.25, #queue-req: 0


[2025-06-18 00:50:13] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 133.20, #queue-req: 0


[2025-06-18 00:50:14] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 133.44, #queue-req: 0
[2025-06-18 00:50:14] INFO:     127.0.0.1:39622 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-06-18 00:50:14] INFO:     127.0.0.1:39624 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-06-18 00:50:14] INFO:     127.0.0.1:39640 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-06-18 00:50:14] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-18 00:50:15] INFO:     127.0.0.1:39652 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-06-18 00:50:15] INFO:     127.0.0.1:39656 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-06-18 00:50:15] Cache flushed successfully!
[2025-06-18 00:50:15] INFO:     127.0.0.1:39660 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-06-18 00:50:15] Start update_weights. Load format=auto
[2025-06-18 00:50:15] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-06-18 00:50:15] Using model weights format ['*.safetensors']


[2025-06-18 00:50:15] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.91it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.91it/s]

[2025-06-18 00:50:15] Update weights end.
[2025-06-18 00:50:15] Cache flushed successfully!
[2025-06-18 00:50:15] INFO:     127.0.0.1:39676 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-06-18 00:50:15] Start update_weights. Load format=auto
[2025-06-18 00:50:15] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-06-18 00:50:16] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-06-18 00:50:16] INFO:     127.0.0.1:39690 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-06-18 00:50:16] Child process unexpectedly failed with exitcode=9. pid=1303180


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-18 00:50:23] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=30929, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=982871380, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, ba

[2025-06-18 00:50:26] Downcasting torch.float32 to torch.float16.


[2025-06-18 00:50:36] Downcasting torch.float32 to torch.float16.


[2025-06-18 00:50:36] Overlap scheduler is disabled for embedding models.
[2025-06-18 00:50:36] Downcasting torch.float32 to torch.float16.
[2025-06-18 00:50:36] Attention backend not set. Use fa3 backend by default.
[2025-06-18 00:50:36] Init torch distributed begin.


[2025-06-18 00:50:37] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-18 00:50:42] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=27.09 GB


[2025-06-18 00:50:43] INFO:     Started server process [1305561]
[2025-06-18 00:50:43] INFO:     Waiting for application startup.
[2025-06-18 00:50:43] INFO:     Application startup complete.
[2025-06-18 00:50:43] INFO:     Uvicorn running on http://0.0.0.0:30929 (Press CTRL+C to quit)


[2025-06-18 00:50:44] INFO:     127.0.0.1:37328 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-18 00:50:44] INFO:     127.0.0.1:37334 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-18 00:50:44] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-18 00:50:45] INFO:     127.0.0.1:37344 - "POST /encode HTTP/1.1" 200 OK
[2025-06-18 00:50:45] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-06-18 00:50:49] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-18 00:50:49] INFO:     127.0.0.1:35198 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-06-18 00:50:49] Child process unexpectedly failed with exitcode=9. pid=1306047
[2025-06-18 00:50:49] Child process unexpectedly failed with exitcode=9. pid=1305899


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-18 00:50:56] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=32438, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=70868255, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downloa

[2025-06-18 00:51:09] Overlap scheduler is disabled for embedding models.
[2025-06-18 00:51:10] Attention backend not set. Use flashinfer backend by default.
[2025-06-18 00:51:10] Init torch distributed begin.


[2025-06-18 00:51:10] Init torch distributed ends. mem usage=0.56 GB


[2025-06-18 00:51:10] Load weight begin. avail mem=46.54 GB


[2025-06-18 00:51:11] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.08it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.07it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.61it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.38it/s]

[2025-06-18 00:51:15] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=31.14 GB, mem usage=15.39 GB.
[2025-06-18 00:51:15] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-06-18 00:51:15] Memory pool end. avail mem=28.35 GB


[2025-06-18 00:51:15] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=27.78 GB


[2025-06-18 00:51:16] INFO:     Started server process [1308573]
[2025-06-18 00:51:16] INFO:     Waiting for application startup.
[2025-06-18 00:51:16] INFO:     Application startup complete.
[2025-06-18 00:51:16] INFO:     Uvicorn running on http://0.0.0.0:32438 (Press CTRL+C to quit)


[2025-06-18 00:51:17] INFO:     127.0.0.1:35844 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-18 00:51:17] INFO:     127.0.0.1:35846 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-18 00:51:17] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-06-18 00:52:20] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-18 00:52:20] INFO:     127.0.0.1:35858 - "POST /encode HTTP/1.1" 200 OK
[2025-06-18 00:52:20] The server is fired up and ready to roll!


[2025-06-18 00:52:34] INFO:     127.0.0.1:46838 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-06-18 00:52:34] Child process unexpectedly failed with exitcode=9. pid=1308813


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-18 00:52:41] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=32209, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=663594514, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, sleep_on_i

[2025-06-18 00:52:52] Attention backend not set. Use flashinfer backend by default.
[2025-06-18 00:52:52] Init torch distributed begin.


[2025-06-18 00:52:53] Init torch distributed ends. mem usage=0.00 GB


[2025-06-18 00:52:54] Load weight begin. avail mem=78.48 GB


[2025-06-18 00:52:54] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-06-18 00:53:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=29.04 GB


[2025-06-18 00:53:01] INFO:     Started server process [1315531]
[2025-06-18 00:53:01] INFO:     Waiting for application startup.
[2025-06-18 00:53:01] INFO:     Application startup complete.
[2025-06-18 00:53:01] INFO:     Uvicorn running on http://0.0.0.0:32209 (Press CTRL+C to quit)


[2025-06-18 00:53:02] INFO:     127.0.0.1:52798 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-18 00:53:02] INFO:     127.0.0.1:52808 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-18 00:53:02] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-18 00:53:03] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1d-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-06-18 00:53:17] Resetting ExpertDistributionRecorder...
[2025-06-18 00:53:17] INFO:     127.0.0.1:52824 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-18 00:53:17] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-18 00:53:19] INFO:     127.0.0.1:52812 - "POST /generate HTTP/1.1" 200 OK
[2025-06-18 00:53:19] The server is fired up and ready to roll!
[2025-06-18 00:53:19] INFO:     127.0.0.1:49896 - "POST /generate HTTP/1.1" 200 OK


[2025-06-18 00:53:19] INFO:     127.0.0.1:49898 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-18 00:53:22] Write expert distribution to /tmp/expert_distribution_recorder_1750208002.0981305.pt
[2025-06-18 00:53:22] Resetting ExpertDistributionRecorder...
[2025-06-18 00:53:22] INFO:     127.0.0.1:49902 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-06-18 00:53:22] Child process unexpectedly failed with exitcode=9. pid=1315950
[2025-06-18 00:53:22] Child process unexpectedly failed with exitcode=9. pid=1315821
